In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
pip freeze > requirements.txt

In [3]:
training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


100%|██████████| 26421880/26421880 [00:02<00:00, 11031658.83it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 174488.75it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3204269.79it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6375044.88it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
from torchvision.transforms import ToPILImage
trans_img=ToPILImage()
pil_img=trans_img(test_data[0][0])
pil_img

In [6]:
batch_size=64
train_dataloader=DataLoader(training_data,batch_size=batch_size)
test_dataloader=DataLoader(test_data,batch_size=batch_size)
for x,y in test_dataloader:
  print(f"Shape of x [N,C,H,W]:{x.shape}")
  print(f"Shape of y:{y.shape}{y.dtype}")
  break

Shape of x [N,C,H,W]:torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64])torch.int64


In [7]:
device=("cuda")
print(f"Using {device} device")

Using cuda device


In [10]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten=nn.Flatten()
    self.linear_relu_stack=nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward(self,x):
    x=self.flatten(x)
    logits=self.linear_relu_stack(x)
    return logits

model=NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-3)

In [13]:
train_dataloader.dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [14]:
def train(dataloader,model,loss_fn,optimizer):
  size=len(dataloader.dataset)
  model.train()
  for batch,(x,y) in enumerate(dataloader):
    x,y=x.to(device),y.to(device)
    pred=model(x)
    loss=loss_fn(pred,y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if batch%100==0:
      loss,current=loss.item(),(batch+1)*len(x)
      print(f"loss:{loss:>7f}[{current:>5d}/{size:>5d}]")



In [15]:
def test(dataloader,model,loss_fn):
  size=len(dataloader.dataset)
  num_batch=len(dataloader)
  model.eval()
  test_loss,correct=0,0
  with torch.no_grad():
    for x,y in dataloader:
      x,y=x.to(device),y.to(device)
      pred=model(x)
      test_loss+=loss_fn(pred,y)
      correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss/=num_batch
    correct/=size
    print(f"Test:\n Accuracy:{(100*correct):>0.1f}%,avg loss:{test_loss:>8f}\n")


In [16]:
epoch=5
for t in range(epoch):
  print(f"Epoch {t+1}\n--------------------")
  train(train_dataloader,model,loss_fn,optimizer)
  test(test_dataloader,model,loss_fn)


Epoch 1
--------------------
loss:2.294363[   64/60000]
loss:2.285391[ 6464/60000]
loss:2.259794[12864/60000]
loss:2.251211[19264/60000]
loss:2.241320[25664/60000]
loss:2.213105[32064/60000]
loss:2.211053[38464/60000]
loss:2.183558[44864/60000]
loss:2.166040[51264/60000]
loss:2.132745[57664/60000]
Test:
 Accuracy:50.5%,avg loss:2.130956

Epoch 2
--------------------
loss:2.137330[   64/60000]
loss:2.128714[ 6464/60000]
loss:2.059985[12864/60000]
loss:2.074912[19264/60000]
loss:2.028999[25664/60000]
loss:1.960593[32064/60000]
loss:1.980349[38464/60000]
loss:1.903635[44864/60000]
loss:1.889098[51264/60000]
loss:1.813556[57664/60000]
Test:
 Accuracy:60.9%,avg loss:1.821681

Epoch 3
--------------------
loss:1.853214[   64/60000]
loss:1.826441[ 6464/60000]
loss:1.695761[12864/60000]
loss:1.739313[19264/60000]
loss:1.647001[25664/60000]
loss:1.591000[32064/60000]
loss:1.612601[38464/60000]
loss:1.528093[44864/60000]
loss:1.538502[51264/60000]
loss:1.433884[57664/60000]
Test:
 Accuracy:62.6%

In [17]:
torch.save(model.state_dict(),"model.pth")

In [18]:
model_test=NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [28]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
model.eval()
x,y=test_data[0][0],test_data[0][1]
with torch.no_grad():
  x=x.to(device)
  pred=model(x)
  predicted,actual=classes[pred.argmax(1).item()],classes[y]
  print(f"Predicted: {predicted}, Actual: {actual}")

Predicted: Ankle boot, Actual: Ankle boot


In [26]:
print(pred.argmax(1).item())
print(pred[0].argmax(0))

9
tensor(9, device='cuda:0')
